In [ ]:
#pragma cling load("libcurl")
#include "./lib/everest.hpp"
#include <list>
using namespace TEMPLET;
using namespace std;

In [ ]:
{
    taskengine eng("login", "pass");
    string token;
    eng.get_access_token(token);
    eng.free_access_token();
    cout <<"access token: "<< token << endl;
    eng.get_app_description("app name(e.g. squared-number)");    
}

In [ ]:
const int NUM_WORKERS = 5;
const int TASK_TIME = 5 * 1000; // ms
const int NUM_TASKS = 10;
double arr[NUM_TASKS];
int cur;

In [ ]:
struct worker;
list<worker*> idle_worker_list;

In [ ]:
struct worker:task {
    /**/
	int pos_in_arr;
	/**/
	worker():task(){}
    
    void on_worker_ready(){
        // put the result
        /**/
        json& out=result();
        arr[pos_in_arr] = out["squared-n"];
        /**/
        
        idle_worker_list.push_back(this);
        run_tasks();
    }
    
    static void run_tasks(){
        // run tasks while having tasks and free workers
        while(!idle_worker_list.empty() && /**/cur < NUM_TASKS/**/){
            worker*w = idle_worker_list.front();
            idle_worker_list.pop_front();

            /**/
            json in;
            in["name"] = "squared-number-task";
            in["inputs"]["n"] = arr[cur];
            in["inputs"]["time"] = TASK_TIME;
            w->input(in);
            w->pos_in_arr = cur;
            cur++;
            /**/
            
            w->submit([=](){w->on_worker_ready();});
        }
    }
};

In [ ]:
taskengine eng("access_token");

worker worker_arr[NUM_WORKERS];
    
for(int i=0;i<NUM_WORKERS;i++){
    worker_arr[i].set_app_id("app_id");
    worker_arr[i].set_engine(eng);
    idle_worker_list.push_back(&worker_arr[i]);
}

/**/
for(int i=0;i<NUM_TASKS;i++){ arr[i] = i; cout << "arr[" << i << "]=" << arr[i] << endl;}
cur = 0;
/**/

worker::run_tasks(); // submitting the first portion of tasks
eng.wait_all();

/**/
cout << "----------\n";
for(int i=0;i<NUM_TASKS;i++)  cout << "arr[" << i << "]=" << arr[i] << endl;;
/**/